<a href="https://colab.research.google.com/github/SantiData87/SantiData87-CoderHouse_DataEngineer_Biblioteca/blob/main/API_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
#Montar en drive
from google.colab import drive
import os
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/'
%ls

Mounted at /content/drive
/content/drive/MyDrive
 219682.pdf                             'Colab Notebooks'/                'Lista Casamiento.xlsx'
 227764.pdf                              CROWDIUM/                         LLC/
 235866.pdf                             '- CUBO MAGICO  - SOLUCION.docx'  '- PEN DRIVE'/
'572 HACER.xlsx'                        '- DATA'/                          profe_solucion_sql.png
 analisis_mercado.txt                   'DE_Indice .gslides'               RUCA.pdf
'- BOLSA'/                               DE_S4_Query_.png                  script.py
'calculadora flujos de pagos ONs.xlsx'  'Documentos Importantes '/        'Semanal 20240315.pdf'
'Calculadora LEDE.xlsx'                  Ej_Paso_a_Paso.gslides            stacked.html
'Ciudadanía Italiana'/                  'Gastos Casamiento.gsheet'         tutor_solucion_sql.png
 client_secret_spotify.txt               HTC/


In [ ]:
# prompt: leer con with open archivo en google drive llamado client_secret_spotify.txt

with open('/content/drive/My Drive/client_secret_spotify.txt', 'r') as f:
  client_secret = f.read()


In [ ]:
#Autenticar
client_id= 'a25133e8be74459883f8c73541945ab9'
#client_secret = '92fc8a22cfe148488360a254966d91e8'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
results = sp.search(q='Los Fundamentalistas del Aire Acondicionado', type='artist', limit=20)
artist_id = results['artists']['items'][0]['id']
artist_id

'1SKA2QwV2fwGgoMlx16FXO'

In [ ]:
albums = sp.artist_albums(artist_id, album_type='album')

for album in albums['items']:
  print(album['name'])

Los Fundamentalistas del Aire Acondicionado en Huracán (En Vivo)
A los Pájaros
Desde los Satélites (En Vivo)
El Ruiseñor, el Amor y la Muerte
En Concierto (En Vivo)
Pajaritos, Bravos Muchachitos
El Perfume de la Tempestad
Porco Rex
El Tesoro de los Inocentes (Bingo Fuel)


In [ ]:
results = sp.search(q='year:2023', type='track', limit=5)
data = {'Id': [],'Artista': [], 'Cancion': [],'Duracion_ms': [], 'Genero': [],'Album': [], 'Album_img': [], 'Total_canciones_album': [], 'Popularidad': [], 'fecha_lanzamiento': []}
for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    duration_ms = track['duration_ms']
    track_id = track['id']
    album_group = track['album']['name']
    album_img = track['album']['images'][0]['url'] #imagen de album
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date']
    #Quitar las comillas
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    #Separar el género por coma
    track_genre = ', '.join(track_genre)

    data['Id'].append(id)
    data['Artista'].append(artist_name)
    data['Cancion'].append(track_name)
    data['Duracion_ms'].append(duration_ms)
    data['Album'].append(album_group)
    data['Album_img'].append(album_img)
    data['Total_canciones_album'].append(album_cont)
    data['Genero'].append(track_genre)
    data['Popularidad'].append(track_popularity)
    data['fecha_lanzamiento'].append(track_year)


df = pd.DataFrame(data)
#Evitar que haya canciones duplicadas
df.drop_duplicates(subset=['Artista', 'Cancion','Album'], keep='first', inplace=True)
#Reemplazar valores nulos o vacios en el campo Género por Desconocido
df['Genero'].fillna('Desconocido', inplace=True)
df.loc[df['Genero'] == '', 'Genero'] = 'Desconocido'
#Evitar que se cargue una canción con duración 0 ms
df = df[df['Duracion_ms'] != 0]
#Verificar que la fecha se muestre en formato fecha
df['fecha_lanzamiento'] = pd.to_datetime(df['fecha_lanzamiento'], format='%Y-%m-%d')
display(df)

,Id,Artista,Cancion,Duracion_ms,Genero,Album,Album_img,Total_canciones_album,Popularidad,fecha_lanzamiento
0,5KsDaCcYp2ebwwYfnxZE77,Jazz For Sleep,Alluring Music for Cocktails after Dinner,111222,Desconocido,Restaurant Relaxation,https://i.scdn.co/image/ab67616d0000b27301f4cc...,40,0,2023-09-11
1,1rHcF8w54nZwOWLdFLQ4ie,enplenafalla,Turosstro..,117330,Desconocido,Turosstro..,https://i.scdn.co/image/ab67616d0000b27394b964...,1,0,2023-12-07
2,6ZHnu5jFDssHMlP6NdTPSs,Mark Chapter,Save Me - Dub Mix,243934,Desconocido,Rated Sounds 2023,https://i.scdn.co/image/ab67616d0000b273408001...,25,0,2023-10-13
3,1yeEcMCr7GHrNHZLo7IHZz,Sweet Lullaby,Energy Replenishment,107023,Desconocido,36 Relaxing Routine,https://i.scdn.co/image/ab67616d0000b2737efaf7...,36,0,2023-02-10
4,7yoq0rGE5itUkoV6BtNo1k,Mind Rest,Newfound Peace,142027,Desconocido,37 Dream Induced Child,https://i.scdn.co/image/ab67616d0000b273ff7d21...,37,0,2023-03-10


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth


# Configura la autenticación
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='a25133e8be74459883f8c73541945ab9',
                                               client_secret='92fc8a22cfe148488360a254966d91e8',
                                               redirect_uri='http//localhost:3000/',
                                               scope='your_scope'))

# Accede a la API usando el token actual
# ...

# Verifica si el token está próximo a expirar
if sp.auth_manager.is_token_expired():
    # Renueva el token de acceso
    sp.auth_manager.refresh_access_token()

# Ahora puedes seguir usando `sp` con el nuevo token renovado


TypeError: SpotifyAuthBase.is_token_expired() missing 1 required positional argument: 'token_info'

In [ ]:
# prompt: obtener de la API de spotify las 10 canciones mas escuchadas por mi

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Autenticar con la API de Spotify
client_id = 'a25133e8be74459883f8c73541945ab9'
client_secret = '92fc8a22cfe148488360a254966d91e8'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Obtener las 10 canciones más escuchadas por el usuario
results = sp.current_user_top_tracks(limit=10)

# Extraer los datos de las canciones
data = []
for track in results['items']:
    data.append({
        'id': track['id'],
        'name': track['name'],
        'artist': track['artists'][0]['name'],
        'album': track['album']['name'],
        'popularity': track['popularity']
    })

# Crear un DataFrame con los datos
df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)


In [ ]:
top_tracks = sp.current_user_top_tracks(limit=10, time_range='long_term')

In [ ]:
# Hacer la solicitud a la API de Spotify para obtener las canciones más escuchadas
top_tracks = sp.current_user_top_tracks(limit=10, time_range='long_term')

# Extraer información relevante de las canciones
top_tracks_info = [(track['name'], ', '.join([artist['name'] for artist in track['artists']])) for track in top_tracks['items']]

# Mostrar las 10 canciones más escuchadas
print("Las 10 canciones más escuchadas por ti son:")
for index, (name, artists) in enumerate(top_tracks_info, start=1):
    print(f"{index}. {name} - {artists}")


In [ ]:
import requests
import json

# Credenciales de la aplicación de Spotify
client_id= 'a25133e8be74459883f8c73541945ab9'
client_secret = '92fc8a22cfe148488360a254966d91e8'

# Endpoint para obtener el token de acceso de Spotify
token_url = 'https://accounts.spotify.com/api/token'

# Datos necesarios para solicitar el token de acceso
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Realizar la solicitud POST para obtener el token de acceso
auth_response = requests.post(token_url, data=auth_data)


# Comprobar si la solicitud fue exitosa (código de estado 200)
if auth_response.status_code == 200:
    # Extraer el token de acceso del cuerpo de la respuesta
    access_token = auth_response.json()['access_token']

    # Endpoint para obtener las canciones más escuchadas del usuario
    endpoint = 'https://api.spotify.com/v1/me/top/tracks'

    # Parámetros de la solicitud
    params = {
        'limit': 10,  # Obtener las 10 canciones principales
        'time_range': 'long_term'  # Periodo de tiempo a considerar (long_term para todo el tiempo)
    }

In [ ]:
import requests
import json

# Credenciales de la aplicación de Spotify
client_id= 'a25133e8be74459883f8c73541945ab9'
client_secret = '92fc8a22cfe148488360a254966d91e8'

# Endpoint para obtener el token de acceso de Spotify
token_url = 'https://accounts.spotify.com/api/token'

# Datos necesarios para solicitar el token de acceso
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Realizar la solicitud POST para obtener el token de acceso
auth_response = requests.post(token_url, data=auth_data)
print(auth_response)


#if auth_response.status_code == 200:
    # Extraer el token de acceso del cuerpo de la respuesta
access_token = auth_response.json()['access_token']

    # Endpoint para obtener las canciones más escuchadas del usuario
endpoint = 'https://api.spotify.com/v1/me/top/tracks'

    # Parámetros de la solicitud
params = {
        'limit': 10,  # Obtener las 10 canciones principales
        'time_range': 'long_term'  # Periodo de tiempo a considerar (long_term para todo el tiempo)
    }

    # Cabeceras de la solicitud con el token de acceso
headers = {
        'Authorization': 'Bearer ' + access_token
    }

    # Realizar la solicitud GET a la API de Spotify
response = requests.get(endpoint, params=params, headers=headers)
print(response)

In [ ]:
import requests
import json

# Credenciales de la aplicación de Spotify
client_id= 'a25133e8be74459883f8c73541945ab9'
client_secret = '92fc8a22cfe148488360a254966d91e8'

# Endpoint para obtener el token de acceso de Spotify
token_url = 'https://accounts.spotify.com/api/token'

# Datos necesarios para solicitar el token de acceso
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Realizar la solicitud POST para obtener el token de acceso
auth_response = requests.post(token_url, data=auth_data)

# Comprobar si la solicitud fue exitosa (código de estado 200)
if auth_response.status_code == 200:
    # Extraer el token de acceso del cuerpo de la respuesta
    access_token = auth_response.json()['access_token']

    # Endpoint para obtener las canciones más escuchadas del usuario
    endpoint = 'https://api.spotify.com/v1/me/top/tracks'

    # Parámetros de la solicitud
    params = {
        'limit': 10,  # Obtener las 10 canciones principales
        'time_range': 'long_term'  # Periodo de tiempo a considerar (long_term para todo el tiempo)
    }

    # Cabeceras de la solicitud con el token de acceso
    headers = {
        'Authorization': 'Bearer ' + access_token
    }

    # Realizar la solicitud GET a la API de Spotify
    response = requests.get(endpoint, params=params, headers=headers)

    # Comprobar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta a formato JSON
        data = response.json()

        # Iterar sobre las canciones y mostrar sus nombres
        print("Las 10 canciones más escuchadas por ti son:")
        for index, track in enumerate(data['items'], start=1):
            print(f"{index}. {track['name']} - {', '.join([artist['name'] for artist in track['artists']])}")
    else:
        print("Error al obtener las canciones más escuchadas:", response.status_code)
else:
    print("Error al obtener el token de acceso:", auth_response.status_code)


In [ ]:
import requests
import json

# Tu token de acceso de Spotify
access_token = 'TU_TOKEN_DE_ACCESO'

# Endpoint para obtener las canciones más escuchadas del usuario
endpoint = 'https://api.spotify.com/v1/me/top/tracks'

# Parámetros de la solicitud
params = {
    'limit': 10,  # Obtener las 10 canciones principales
    'time_range': 'long_term'  # Periodo de tiempo a considerar (long_term para todo el tiempo)
}

# Cabeceras de la solicitud con el token de acceso
headers = {
    'Authorization': 'Bearer ' + access_token
}

# Realizar la solicitud GET a la API de Spotify
response = requests.get(endpoint, params=params, headers=headers)

# Comprobar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Convertir la respuesta a formato JSON
    data = response.json()

    # Iterar sobre las canciones y mostrar sus nombres
    print("Las 10 canciones más escuchadas por ti son:")
    for index, track in enumerate(data['items'], start=1):
        print(f"{index}. {track['name']} - {', '.join([artist['name'] for artist in track['artists']])}")
else:
    print("Error al obtener las canciones más escuchadas:", response.status_code)
